# Master Script: HAPI Prediction Pipeline (RF + XGBoost + SHAP)

## Overview
This is the Master Script for the "Beyond Braden" project. It consolidates the entire analytic pipeline into a single execution flow. It takes the final merged clinical dataset and performs cleaning, leakage detection, model training, evaluation, and patient risk stratification.

## Features
1.  **Automated Leakage Detection:** * Scans all features for correlation $> 0.95$ with the target.
    * Automatically drops "future" columns (e.g., *Length of Stay*, *Wound Vac usage*) to ensure the model predicts risk rather than confirming a diagnosis.
2.  **Model Competition:** * Trains a **Random Forest** (Baseline).
    * Trains an **XGBoost** classifier (Advanced).
3.  **Explainable AI (SHAP):** generates Beeswarm and Bar plots to explain *why* the model flags specific patients.
4.  **Clinical Decision Support:** Outputs a prioritized list of "High Risk" patients for clinical review.

## Requirements
* **Python 3.8+**
* **Libraries:** `pandas`, `numpy`, `scikit-learn`, `xgboost`, `shap`, `matplotlib`, `seaborn`

```bash
pip install scikit-learn xgboost shap pandas matplotlib seaborn

In [32]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
    accuracy_score
)
from xgboost import XGBClassifier

In [33]:
# Configure
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
    accuracy_score
)
from xgboost import XGBClassifier

In [34]:
# Configure
# Path to the final merged analytic dataset (created in previous steps)
DATA_PATH = r"D:\School\5141\FINAL_HAPI_ANALYTIC.csv"

# The binary target variable (1 = HAPI, 0 = No HAPI)
TARGET_COL = "HAPI_FINAL"

# Output directory for PNGs, CSVs, and risk tables
OUTPUT_DIR = r"D:\School\5141\model_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [35]:
# Data Cleaning and Leakage Prevention
# These columns contain information that happens AFTER the outcome or are proxy labels.
# We must drop them to prevent Data Leakage.

BAD_COLUMNS = [
    # Direct labels or synonymous outcomes
    "HAPI_STRUCTURED",
    "HAPI_UNSTRUCTURED",
    "has_HAPI",
    "notes_pu",
    
    # Treatment that happens after a HAPI develops
    "has_wound_vac",
    "first_wound_note_time",

    # Time-dependent variables
    # LOS is often longer *because* they got a HAPI, not the cause.
    "LOS_DAYS",
    "LOS_HOURS",
    "hours_after_admit",

    # Summary stats that aggregate the entire stay 
    "num_distinct_drugs",
    "num_distinct_pharm_meds",
    "num_pharm_orders",
    "total_meds_orders",
    "num_prescriptions",
    "num_high_risk_drugs",
    "num_high_risk_pharm_orders",
    "has_sedation_drug",
    "has_high_risk_drug",
    "has_high_risk_pharm_order",

    # ICU procedures over entire stay
    "had_icu_procedure",
    "had_icu_procedure_0_24h",
    "had_icu_procedure_0_48h",
    "num_icu_procedures_0_24h",
    "num_icu_procedures_0_48h",
]


In [36]:
# Labels 
# These columns contain information that happens after the outcome or are proxy labels.
# We must drop them to prevent "Data Leakage."

BAD_COLUMNS = [
    # Direct labels or synonymous outcomes
    "HAPI_STRUCTURED",
    "HAPI_UNSTRUCTURED",
    "has_HAPI",
    "notes_pu",
    
    # Treatment that happens *after* a HAPI develops
    "has_wound_vac",
    "first_wound_note_time",

    # Time-dependent variables (Post-Outcome Info)
    # LOS is often longer *because* they got a HAPI, not the cause.
    "LOS_DAYS",
    "LOS_HOURS",
    "hours_after_admit",

    # Summary stats that aggregate the *entire* stay (Future info)
    "num_distinct_drugs",
    "num_distinct_pharm_meds",
    "num_pharm_orders",
    "total_meds_orders",
    "num_prescriptions",
    "num_high_risk_drugs",
    "num_high_risk_pharm_orders",
    "has_sedation_drug",
    "has_high_risk_drug",
    "has_high_risk_pharm_order",

    # ICU procedures over entire stay
    "had_icu_procedure",
    "had_icu_procedure_0_24h",
    "had_icu_procedure_0_48h",
    "num_icu_procedures_0_24h",
    "num_icu_procedures_0_48h",
]


In [37]:
# Visualization Labels 
# Mapping column names to clean, human-readable labels for charts.
FEATURE_LABELS = {
    "age": "Age",
    "anchor_age": "Age at Admission",
    "gender_F": "Female Gender",
    "gender_M": "Male Gender",
    "bmi": "Body Mass Index",
    "icu_stay": "ICU Stay Indicator (Early)",
    "icu_hours_0_48h": "ICU Hours (First 48h)",
    "num_lab_events_0_48h": "Lab Events (First 48h)",
    "avg_heart_rate_0_48h": "Average Heart Rate (First 48h)",
    "min_map_0_48h": "Lowest MAP (First 48h)",
    "max_map_0_48h": "Highest MAP (First 48h)",
    "avg_resp_rate_0_48h": "Average Respiratory Rate (First 48h)",
    "avg_temp_0_48h": "Average Temperature (First 48h)",
    "avg_spo2_0_48h": "Average SpO₂ (First 48h)",
    "num_procedures": "Number of Procedures",
    "num_med_admins_0_48h": "Medication Administrations (First 48h)",
    "creatinine_first": "Creatinine (First Lab)",
    "glucose_first": "Glucose (First Lab)",
    "albumin_first": "Albumin (First Lab)",
    "gcs_total_0_24h": "GCS Total (First 24h)",
    "rrt": "Renal Replacement Therapy",
    "num_ed_visits_lastyear": "ED Visits (Last Year)",
    "num_admissions_lastyear": "Admissions (Last Year)",
}

def label_feature(name: str) :
    return FEATURE_LABELS.get(name, name.replace("_", " ").title())


In [38]:

def remove_leakage(X: pd.DataFrame, y: pd.Series, threshold: float = 0.95):
    """
    Scans for features that are statistically too similar to the target.
    If correlation > 0.95, it's likely a proxy for the label (Leakage).
    """
    y_numeric = y.astype(float).values
    leaky_cols = []

    for col in X.columns:
        col_values = X[col].values.astype(float)
        if np.std(col_values) == 0: continue # Skip constant columns
        
        corr = np.corrcoef(col_values, y_numeric)[0, 1]
        if np.isnan(corr): continue
        
        if abs(corr) >= threshold:
            leaky_cols.append((col, corr))

    if leaky_cols:
        for col, corr in leaky_cols:
            print(f"   - {col}: correlation = {corr:.3f}")
        X = X.drop(columns=[c for c, _ in leaky_cols])
    else:
        print("No additional high-correlation leakage detected.")

    return X

In [39]:
def load_data(path: str, target_col: str):
    """
    Master data loader. Performs data cleaning process:
    1. Load CSV
    2. Drop ID columns
    3. Drop known 'BAD_COLUMNS' (defined above)
    4. Drop non-numeric data
    5. Fill NaNs
    6. Drop columns that match the target exactly
    7. Run statistical leakage check
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"CSV not found at: {path}")

    df = pd.read_csv(path)

    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found.")

    y = df[target_col].astype(int)

    # Drop IDs, Target, and Known Bad Columns
    drop_cols = ["hadm_id", "subject_id", target_col]
    
    # Drop columns with suspicious names 
    leak_patterns = ["HAPI", "ULCER", "PRESSURE_ULCER", "PU_LABEL", "WOUND"]
    for col in df.columns:
        upper = col.upper()
        if any(pat in upper for pat in leak_patterns) and col != target_col:
            drop_cols.append(col)

    drop_cols.extend(BAD_COLUMNS)
    
    # Intersect with actual columns to avoid errors
    drop_cols = list(set(c for c in drop_cols if c in df.columns))
    print(f"Dropping {len(drop_cols)} columns (IDs, Target, Leakage).")
    X = df.drop(columns=drop_cols)

    # Drop non-numeric (Text/Dates)
    non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        print(f"Dropping non-numeric columns: {non_numeric}")
        X = X.drop(columns=non_numeric)

    # Fill Missing Values (0 is standard for counts/flags)
    X = X.fillna(0)

    # Drop "Perfect Predictors" (Columns that are exactly the label)
    leaky_exact = []
    for col in X.columns:
        if X[col].nunique() <= 5:
            if (X[col] == y).all() or (X[col] == 1 - y).all():
                leaky_exact.append(col)
    
    if leaky_exact:
        print(f"Dropping exact label duplicates: {leaky_exact}")
        X = X.drop(columns=leaky_exact)

    # Statistical Leakage Check
    X = remove_leakage(X, y, threshold=0.95)

    print(f"\nFinal Dataset: {X.shape[0]} rows, {X.shape[1]} features.")
    print("Class Balance (Target):")
    print(y.value_counts(normalize=True).rename("proportion"))

    return X, y


In [40]:
def evaluate_model(model_name, y_true, y_pred, y_prob):
    """Generates a DataFrame row of metrics (Precision, Recall, F1, AUC)."""
    report = classification_report(y_true, y_pred, output_dict=True)
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)

    rows = []
    for label, stats in report.items():
        if isinstance(stats, dict):
            rows.append({
                "model": model_name,
                "label": label,
                "precision": stats.get("precision", np.nan),
                "recall": stats.get("recall", np.nan),
                "f1": stats.get("f1-score", np.nan),
                "support": stats.get("support", np.nan),
                "roc_auc": auc,
                "accuracy": acc,
            })
    return pd.DataFrame(rows)

def confusion_matrix_output(y_true, y_pred, title, save_path=None, normalize=False):
    """Plots a standard Seaborn heatmap for the confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1, keepdims=True)
        fmt = ".2f"
    else:
        fmt = "d"

    labels = ["No HAPI", "HAPI"]
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap="rocket", xticklabels=labels, yticklabels=labels, cbar=False)
    plt.title(title)
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"Saved plot: {save_path}")
    plt.close()

def humanize_text(importances: pd.Series, title: str, filename: str):
    """
    Maps raw feature names to human-readable labels using FEATURE_LABELS
    and saves a horizontal bar chart.
    """
    top20 = importances.head(20)
    pretty_index = [label_feature(col) for col in top20.index]
    pretty_series = pd.Series(top20.values, index=pretty_index)

    print(f"\n{title} (Top 5):")
    print(pretty_series.head(5).to_string())

    plt.figure(figsize=(10, 8))
    # Invert so top feature is at the top of the chart
    pretty_series.iloc[::-1].plot(kind="barh", color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel("Importance Score")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

    
def summarize_patient_risk(model, X_subset, y_subset=None, model_name="Model", top_n=10):
    """
    Generates a Clinical Risk List. 
    Returns the top N patients most likely to have HAPI based on the model.
    """
    probs = model.predict_proba(X_subset)[:, 1]
    
    df = pd.DataFrame({"predicted_risk": probs}, index=X_subset.index)
    if y_subset is not None:
        df["true_label"] = y_subset

    # Risk Stratification 
    def bucket(p):
        if p >= 0.40: return "HIGH"
        elif p >= 0.20: return "MEDIUM"
        else: return "LOW"

    df["risk_bucket"] = df["predicted_risk"].apply(bucket)
    df_sorted = df.sort_values("predicted_risk", ascending=False).head(top_n)

    print(f"\n--- Top {top_n} High-Risk Patients ({model_name}) ---")
    print(df_sorted)
    return df_sorted


In [41]:
#Execute

if __name__ == "__main__":
    
    # Load Data    
    X, y = load_data(DATA_PATH, TARGET_COL)
    
    # Stratified split to ensure HAPI cases are distributed evenly
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"\nTrain shape: {X_train.shape} | Test shape: {X_test.shape}")

    all_metrics = []

    # Basline Model: Random Forest
    print("\n" + "-"*60)
    print("TRAINING: Random Forest (Baseline)")
    print("-"*60)

    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        n_jobs=-1,
        random_state=42,
        class_weight=None 
    )
    rf.fit(X_train, y_train)

    # Predictions
    rf_pred = rf.predict(X_test)
    rf_prob = rf.predict_proba(X_test)[:, 1]

    # Evaluate
    print("\nRandom Forest Report:")
    print(classification_report(y_test, rf_pred))
    all_metrics.append(evaluate_model("RandomForest", y_test, rf_pred, rf_prob))

    # Save Confusion Matrix
    confusion_matrix_output(
        y_test, rf_pred, 
        title="Random Forest - Confusion Matrix",
        save_path=os.path.join(OUTPUT_DIR, "rf_confusion_matrix.png"),
        normalize=True
    )
    
    # Feature Importance 
    rf_imps = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    humanize_text(rf_imps, "Top 20 Features - Random Forest", "rf_top20_features.png")


    # Model: XGBOOST
    print("\n" + "-"*60)
    print("TRAINING: XGBoost (Gradient Boosting)")
    print("-"*60)
    xgb = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist", 
        eval_metric="auc",
        random_state=42,
        n_jobs=-1
    )
    xgb.fit(X_train, y_train)

    # Predictions
    xgb_pred = xgb.predict(X_test)
    xgb_prob = xgb.predict_proba(X_test)[:, 1]

    # Evaluate
    print("\nXGBoost Report:")
    print(classification_report(y_test, xgb_pred))
    all_metrics.append(evaluate_model("XGBoost", y_test, xgb_pred, xgb_prob))

    # Save Confusion Matrix
    confusion_matrix_output(
        y_test, xgb_pred, 
        title="XGBoost - Confusion Matrix",
        save_path=os.path.join(OUTPUT_DIR, "xgb_confusion_matrix.png"),
        normalize=True
    )

    # Feature Importance (XGB)
    xgb_imps = pd.Series(xgb.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    humanize_text(xgb_imps, "Top 20 Features - XGBoost", "xgb_top20_features.png")


    # SHAP ANALYSIS: explains why the model made specific decisions.
    # It takes a sample of the test set to speed up calculation.
    print("\n" + "-"*60)
    print("EXPLAINABILITY: Computing SHAP Values")
    print("-"*60)
    
    shap_sample = X_test.sample(n=2000, random_state=42) if len(X_test) > 2000 else X_test
    explainer = shap.TreeExplainer(xgb)
    shap_values = explainer.shap_values(shap_sample)

    # Bar Plot 
    plt.figure()
    shap.summary_plot(shap_values, shap_sample, plot_type="bar", max_display=20, show=False)
    plt.title("SHAP Feature Importance (XGBoost)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "xgb_shap_bar.png"), dpi=300)
    plt.close()

    # Beeswarm Plot 
    plt.figure()
    shap.summary_plot(shap_values, shap_sample, max_display=20, show=False)
    plt.title("SHAP Beeswarm Summary (XGBoost)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "xgb_shap_beeswarm.png"), dpi=300)
    plt.close()
    
    print(f"   > SHAP plots saved to {OUTPUT_DIR}")


    # Patient Risk Stratification
    # Identify specific patients who need intervention
    _ = summarize_patient_risk(xgb, X_test, y_test, model_name="XGBoost", top_n=10)

    # Save Results
    metrics_df = pd.concat(all_metrics, ignore_index=True)
    metrics_path = os.path.join(OUTPUT_DIR, "model_comparison_metrics.csv")
    metrics_df.to_csv(metrics_path, index=False)
    
    print(f"\nPipeline Complete. All outputs saved to: {OUTPUT_DIR}")

Dropping 26 columns (IDs, Target, Leakage).
Dropping non-numeric columns: ['admittime']
No additional high-correlation leakage detected.

Final Dataset: 555244 rows, 61 features.
Class Balance (Target):
HAPI_FINAL
0    0.744428
1    0.255572
Name: proportion, dtype: float64

Train shape: (444195, 61) | Test shape: (111049, 61)

------------------------------------------------------------
TRAINING: Random Forest (Baseline)
------------------------------------------------------------

Random Forest Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87     82668
           1       0.62      0.53      0.57     28381

    accuracy                           0.80    111049
   macro avg       0.74      0.71      0.72    111049
weighted avg       0.79      0.80      0.79    111049

Saved plot: D:\School\5141\model_outputs\rf_confusion_matrix.png

Top 20 Features - Random Forest (Top 5):
Num Detail Items     0.143332
Age                  0.142